In [1]:
import matplotlib.pyplot as plt
#import keras
import pandas as pd
import numpy as np
import pymysql
import tensorflow as tf
import mysql.connector
import datetime
import os

from mysql.connector import Error 
from sklearn.preprocessing import MinMaxScaler

In [2]:
#TAKE USER INPUT
print("Enter the Stock Name from the below list \n")
print(os.listdir("C:\\Users\\Dell\\DatabaseJcomponent\\stockpredictionenv\\data"))
print("\n")
try:
    stock_name = input("Enter the Only Stock Name")
    df=pd.read_csv("data/"+stock_name+".csv")    
    print("\n---Data before the pre processing---\n")
    print('Number of rows and columns:', df.shape)
    print(df.head(5))
    
    #Check the the any row cotain the null value or not 
    print(df.isnull().sum())
    
    #if row contain the null value then drop the row 
    processed_data = df.dropna()
    print(processed_data.isnull().sum())
    
except OSError as e:
    print("Data not avaliable for "+stock_name+". please Enter the valid stock name.")



Enter the Stock Name from the below list 

['AAPL.csv', 'GOOG.csv', 'TATAMOTORS.csv', 'TSLA.csv']


Enter the Only Stock NameTATAMOTORS

---Data before the pre processing---

Number of rows and columns: (7534, 7)
         Date     Open       High      Low      Close  Adj Close  Volume
0  1991-01-02  20.9596  21.857901  20.9596  21.857901   4.031160     0.0
1  1991-01-03  20.9596  21.857901  20.9596  21.857901   4.031160     0.0
2  1991-01-04      NaN        NaN      NaN        NaN        NaN     NaN
3  1991-01-07  20.3608  21.259001  20.0613  21.109301   3.893099     0.0
4  1991-01-08      NaN        NaN      NaN        NaN        NaN     NaN
Date           0
Open         374
High         374
Low          374
Close        374
Adj Close    374
Volume       374
dtype: int64
Date         0
Open         0
High         0
Low          0
Close        0
Adj Close    0
Volume       0
dtype: int64


In [3]:
def dataFrameToMySQL(processed_data):
    try:
        conn = pymysql.connect(host='localhost',port=int(3306),user='root',passwd='root',db='databasejcomponent')
        cur = conn.cursor()
        cur.execute("delete from datatable")
        conn.commit()
        
        for(row,rs) in processed_data.iterrows():
            now = rs[0]
            Open = str(float(rs[1]))
            High = str(float(rs[2]))
            Low = str(float(rs[3]))
            Close = str(float(rs[4]))
            AdjClose = str(float(rs[5]))  
            Volume = str(float(rs[6]))
            insert_query = "insert into datatable values('"+ now +"','"+ Open +"','"+ High +"','"+ Low +"','"+ Close +"','"+ AdjClose +"','"+ Volume +"')"
            cur.execute(insert_query)
        conn.commit()
        cur.close()
    except Error as e:
        print("Error in MySQL connection : ",e)
    finally:
        conn.close()

dataFrameToMySQL(processed_data)

In [4]:
conn = pymysql.connect(host='localhost',port=int(3306),user='root',passwd='root',db='databasejcomponent')
df1 = pd.read_sql('SELECT * FROM datatable', con=conn)
print("\n---Data before the pre processing---\n")
print('Number of rows and columns:',df1.shape)
df1.head(5)


---Data before the pre processing---

Number of rows and columns: (7160, 7)


,Date,Open,High,Low,Close,Adj Close,Volume
0,1991-01-02,20.959600,21.857901,20.9596,21.857901,4.031160,0
1,1991-01-03,20.959600,21.857901,20.9596,21.857901,4.031160,0
2,1991-01-07,20.360800,21.259001,20.0613,21.109301,3.893099,0
3,1991-01-09,21.259001,21.259001,20.0613,20.510500,3.782665,0
4,1991-01-11,20.360800,20.959600,20.0613,20.959600,3.865490,0


In [5]:
training_set = df1.iloc[:800, 4:5].values
test_set = df1.iloc[800:, 4:5].values

In [6]:
# Feature Scaling
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

# Creating a data structure with 50 time-steps and 1 output
X_train = []
y_train = []

for i in range(60, 800):
    X_train.append(training_set_scaled[i-50:i, 0])
    y_train.append(training_set_scaled[i, 0])
X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
print(X_train)

[[[0.04683209]
  [0.05234132]
  [0.05234132]
  ...
  [0.17906273]
  [0.17906273]
  [0.17906273]]

 [[0.05234132]
  [0.05234132]
  [0.05234132]
  ...
  [0.17906273]
  [0.17906273]
  [0.17906273]]

 [[0.05234132]
  [0.05234132]
  [0.04132286]
  ...
  [0.17906273]
  [0.17906273]
  [0.1735535 ]]

 ...

 [[0.83471221]
  [0.89531742]
  [0.89807196]
  ...
  [0.82920116]
  [0.79614393]
  [0.78512555]]

 [[0.89531742]
  [0.89807196]
  [0.91735619]
  ...
  [0.79614393]
  [0.78512555]
  [0.82920116]]

 [[0.89807196]
  [0.91735619]
  [0.92837646]
  ...
  [0.78512555]
  [0.82920116]
  [0.84022144]]]
